# Functionality 1 - Find the Neighbours!

Take input:

a node v

One of the following distances function: 

t(x,y), d(x,y) or network distance .

a distance threshold d.

Implement an algorithm (using proper data structures) that returns :

the set of nodes at distance <= d from v, corresponding to v’s neighborhood.

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np

# Read DS

# Distance :

In [2]:
df_D = pd.read_csv('./DATA/Distance_graph',header=0, sep=',',
                index_col=None, encoding = "ISO-8859-1")
df_D.columns=["Distance"]
df_D.drop([0,1,2,3,4,5],inplace=True)
df_D.reset_index(drop=True)
df_D.head(10)


,Distance
6,a 1 1048577 456
7,a 1048577 1 456
8,a 2 1048578 2389
9,a 1048578 2 2389
10,a 3 1048579 358
11,a 1048579 3 358
12,a 4 1048579 14901
13,a 1048579 4 14901
14,a 5 6 18133
15,a 6 5 18133


In [3]:
df_D=df_D['Distance'].str.split(" " , expand=True)
df_D.columns=['a','Node1','Node2','Distance']
df_D['Node1']=pd.to_numeric(df_D['Node1'])
df_D['Node2']=pd.to_numeric(df_D['Node2'])
df_D['Distance']=pd.to_numeric(df_D['Distance'])
df_D=df_D.reset_index(drop=True)
df_D.head(4)

,a,Node1,Node2,Distance
0,a,1,1048577,456
1,a,1048577,1,456
2,a,2,1048578,2389
3,a,1048578,2,2389


In [4]:
df_D.shape

(4657742, 4)

In [5]:
df_T = pd.read_csv('./DATA/Travel_time_graph',header=0, sep=',',
                index_col=None, encoding = "ISO-8859-1")
df_T.columns=["Distance"]
df_T.drop([0,1,2,3,4,5],inplace=True)
df_T.head(5)

,Distance
6,a 1 1048577 1139
7,a 1048577 1 1139
8,a 2 1048578 5972
9,a 1048578 2 5972
10,a 3 1048579 895


In [6]:
df_T=df_T["Distance"].str.split(" " , expand=True)
df_T.columns=['a','Node1','Node2','Distance']
df_T['Node1']=pd.to_numeric(df_T['Node1'])
df_T['Node2']=pd.to_numeric(df_T['Node2'])
df_T['Distance']=pd.to_numeric(df_T['Distance'])
df_T=df_T.reset_index(drop=True)
df_T.head(5)

,a,Node1,Node2,Distance
0,a,1,1048577,1139
1,a,1048577,1,1139
2,a,2,1048578,5972
3,a,1048578,2,5972
4,a,3,1048579,895


In [7]:
df_T.shape

(4657742, 4)

# Search Neighbours:


In [49]:
node= int(input("Enter node numbner "))
distance_type=input("Enter one of this Distance type: Time or Physical ")
threshold=int(input("Enter your threshold "))  
#The idea is to iterate all direct neighbour from node v
#and check the values Corresponding to the input
#and we recursively subtract the new distance from initial threshold 
#to check the neighbours of the initial node's neighbour's Satisfying the threshold
def search_Neighbours(node, distance_type, threshold): 
    if distance_type == "Physical":
        data = df_D
    elif distance_type == "Time":
        data=df_T   
    out_df = data[(data['Node1'] == node) & (data['Distance'] <= threshold)]

    Neighbours=list(out_df.Node2.values)
        #print(Neighbours)
    for j in Neighbours:
        neigh_dis = data[(data['Node1'] == node) & (data['Node2'] == j)]
        neigh_dis=int(neigh_dis['Distance'])
        out_df=pd.concat([out_df,search_Neighbours(j, distance_type, threshold-neigh_dis)])
    return out_df

out_df=search_Neighbours(node,distance_type,threshold)
out_df.head(7)

Enter node numbner 300
Enter one of this Distance type: Time or Physical Physical
Enter your threshold 5000


,a,Node1,Node2,Distance
1003,a,300,1048816,1018
1008,a,300,1048818,597
1002,a,1048816,300,1018
1071,a,1048816,1048832,365
1003,a,300,1048816,1018
1008,a,300,1048818,597
1002,a,1048816,300,1018


In [50]:
all_Neighbours = list(set(out_df['Node2'].values))
all_Neighbours


[1048832,
 228,
 1048836,
 300,
 303,
 1048816,
 2448,
 1048818,
 1048819,
 1048817,
 1048820]

# The map 

In [52]:
import folium
from folium import plugins
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [53]:
df_c = pd.read_csv('./DATA/Coordinates',header=0, sep=',',
                index_col=None, encoding = "ISO-8859-1")
df_c.columns=["Coordinates"]
df_c.drop([0,1,2,3,4,5],inplace=True)
df_c.head(3)

,Coordinates
6,v 1 -114315309 34133550
7,v 2 -114223946 34176221
8,v 3 -114307299 34148791


In [54]:
df_c=df_c.Coordinates.str.split(" " , expand=True)
df_c.columns=['v','Node_ID','long','lat']
df_c['Node_ID']=pd.to_numeric(df_c['Node_ID'])
df_c['long']=pd.to_numeric(df_c['long'])
df_c['lat']=pd.to_numeric(df_c['lat'])
df_c=df_c.reset_index(drop=True)
df_c.head(5)

,v,Node_ID,long,lat
0,v,1,-114315309,34133550
1,v,2,-114223946,34176221
2,v,3,-114307299,34148791
3,v,4,-114318765,34138889
4,v,5,-114347300,34042614


In [55]:
df_c['lat']=df_c['lat']/1000000
df_c['long']=df_c['long']/1000000

In [56]:
df_c.head(4)

,v,Node_ID,long,lat
0,v,1,-114.315309,34.133550
1,v,2,-114.223946,34.176221
2,v,3,-114.307299,34.148791
3,v,4,-114.318765,34.138889


In [57]:
out_df_cor=pd.merge(df_c,out_df, right_on="Node2", left_on='Node_ID' )

In [58]:
out_df_cor.head(5)

,v,Node_ID,long,lat,a,Node1,Node2,Distance
0,v,228,-114.033229,33.953738,a,1048832,228,1957
1,v,228,-114.033229,33.953738,a,1048832,228,1957
2,v,228,-114.033229,33.953738,a,1048832,228,1957
3,v,228,-114.033229,33.953738,a,1048832,228,1957
4,v,300,-114.030468,33.955685,a,1048816,300,1018


In [59]:
out_df_cor=out_df_cor.drop_duplicates()

In [60]:
out_df_cor

,v,Node_ID,long,lat,a,Node1,Node2,Distance
0,v,228,-114.033229,33.953738,a,1048832,228,1957
4,v,300,-114.030468,33.955685,a,1048816,300,1018
7,v,300,-114.030468,33.955685,a,1048818,300,597
31,v,303,-114.029393,33.956453,a,1048818,303,711
45,v,303,-114.029393,33.956453,a,1048819,303,1577
47,v,2448,-114.033365,33.956606,a,1048832,2448,2514
49,v,1048816,-114.031306,33.955088,a,300,1048816,1018
51,v,1048816,-114.031306,33.955088,a,1048832,1048816,365
79,v,1048817,-114.028244,33.957519,a,1048819,1048817,205
88,v,1048818,-114.029977,33.956035,a,300,1048818,597


In [61]:
lat=list(df_c['lat'].values)
long=list(df_c['long'].values)

In [75]:
m = folium.Map(location=[np.mean(lat),np.mean(long)],zoom_start=6)

In [76]:
m

In [77]:
# plotting the starting node and node within the threshold
for i in range(len(out_df_cor)):
    folium.CircleMarker(location=[lat[i],long[i]],
                        radius=5,
                        popup=['Node_ID'],
                        fill_color="#3db7e4",
                       ).add_to(m)

In [78]:
m.save('index.html')
m